<a href="https://colab.research.google.com/github/racheltr248/Amazon_Vine_Analysis/blob/main/vine_review_analysis_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deliverable 2

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-10 00:57:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.51MB/s    in 0.2s    

2022-10-10 00:57:28 (5.51 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_review_analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load vine_table.csv from S3 bucket
from pyspark import SparkFiles
url = "https://racheltr-bucket.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [5]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



Step 1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [8]:
# create dataframe
df_reviews = df.select(['star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
df_reviews.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          1|   N|                Y|
|          3|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          1|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                N|
|          1|            0|          0|   N|                Y|
|          2|            0|          0|   N|           

In [9]:
# filter for rows where total votes >= 20
df_reviews_20 = df_reviews.filter(df_reviews['total_votes'] >= 20)
df_reviews_20.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          1|          147|        175|   N|                Y|
|          1|           14|         31|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          4|           31|         36|   N|                N|
|          2|           19|         34|   N|                Y|
|          5|           28|         31|   N|                N|
|          1|            4|         32|   N|                N|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
|          1|            5|         36|   N|           

Step 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [10]:
# filter for rows where helpful vote >= .50
df_reviews_50 = df_reviews_20.filter(df_reviews_20['helpful_votes'] / df_reviews_20['total_votes'] >= .50)
df_reviews_50.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          1|          147|        175|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          4|           31|         36|   N|                N|
|          2|           19|         34|   N|                Y|
|          5|           28|         31|   N|                N|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
|          1|           23|         28|   N|                N|
|          5|           88|        110|   N|                N|
|          2|           99|        119|   N|           

Step 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [15]:
# filter for rows where review was part of vine program
df_paid_reviews = df_reviews_50.filter(df_reviews_50['vine'] == 'Y')
df_paid_reviews.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           56|         63|   Y|                N|
|          3|           23|         23|   Y|                N|
|          5|           19|         24|   Y|                N|
|          4|           22|         26|   Y|                N|
|          3|           20|         26|   Y|                N|
|          5|           46|         51|   Y|                N|
|          5|           22|         28|   Y|                N|
|          5|           21|         24|   Y|                N|
|          4|           21|         28|   Y|                N|
|          5|           17|         20|   Y|                N|
|          5|           55|         58|   Y|                N|
|          5|           35|         41|   Y|                N|
|          5|           18|         20|   Y|           

Step 4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [30]:
# filter for rows where reviews were not part of vine program
df_unpaid_reviews = df_reviews_50.filter(df_reviews_50['vine'] == 'N')
df_unpaid_reviews.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          1|          147|        175|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          4|           31|         36|   N|                N|
|          2|           19|         34|   N|                Y|
|          5|           28|         31|   N|                N|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
|          1|           23|         28|   N|                N|
|          5|           88|        110|   N|                N|
|          2|           99|        119|   N|           

In [17]:
# look at summary statistics
df_paid_reviews.describe().show()
df_unpaid_reviews.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|                94|               94|                94|  94|               94|
|   mean| 4.202127659574468|54.59574468085106|61.787234042553195|null|             null|
| stddev|0.9791348741656415|65.26098459822536| 68.90976994895392|null|             null|
|    min|                 1|               14|                20|   Y|                N|
|    max|                 5|              347|               362|   Y|                N|
+-------+------------------+-----------------+------------------+----+-----------------+

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+-------

Step 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

Vine (paid) Reviews

In [26]:
# determine count of reviews in vine program
paid_reviews = df_paid_reviews.count()
paid_reviews

94

In [27]:
# determine count of 5-star reviews in vine program
paid_5star = df_paid_reviews[df_paid_reviews['star_rating'] == 5].count()
paid_5star

48

In [28]:
# determine percent of 5-star reviews in vine program
percent_5star_vine = paid_5star / paid_reviews * 100
percent_5star_vine

51.06382978723404

Non-Vine (unpaid) Reviews

In [31]:
# determine count of reviews not in vine program
unpaid_reviews = df_unpaid_reviews.count()
unpaid_reviews

40471

In [32]:
# determine count of 5-star reviews not in vine program
unpaid_5star = df_unpaid_reviews[df_unpaid_reviews['star_rating'] == 5].count()
unpaid_5star

15663

In [33]:
# determine percent of 5-star reviews not in vine program
percent_5star_unpaid = unpaid_5star / unpaid_reviews * 100
percent_5star_unpaid

38.701786464381904